In [ ]:
import inspect
from azureml.core import Workspace, Dataset
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.pipeline.core.graph import DataType
from azureml.pipeline.wrapper import Module, dsl, Pipeline
from azureml.core.authentication import InteractiveLoginAuthentication

In [ ]:
subscription_id = '4f455bd0-f95a-4b7d-8d08-078611508e0b'
resource_group = 'fundamental'
workspace_name = 'fundamental4'
namespace=workspace_name # for loading module
# set this if you have multiple tenant
tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47"
interactive_auth = InteractiveLoginAuthentication(tenant_id=tenant_id)

workspace = Workspace(subscription_id, resource_group, workspace_name)
print(workspace.name, workspace.resource_group, workspace.location, workspace.subscription_id, workspace.compute_targets.keys(),sep = '\n')

In [ ]:
# choose compute target
print(workspace.compute_targets)
aml_compute_name = 'aml-compute'
try:
    aml_compute = AmlCompute(workspace, aml_compute_name)
    print("Found existing compute target: {}".format(aml_compute_name))
except:
    print("Creating new compute target: {}".format(aml_compute_name))

    provisioning_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                                min_nodes=1,
                                                                max_nodes=4)
    aml_compute = ComputeTarget.create(workspace, aml_compute_name, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
# print(aml_compute)

In [ ]:
# register my own datatype
DataType.create_data_type(workspace, 'MyDirectory', description='', is_directory=True) # won't register repeatedly
DataType.create_data_type(workspace, 'MyFile', description='', is_directory=False)

In [ ]:
# load data
dataset_name = 'THUCNews_TXT'
char2index_name = 'Char2Index_JSON'

if dataset_name not in workspace.datasets:
    print('Registering a THUCNews dataset for fasttext pipeline ...')
    path = ['https://datastore4fasttext.file.core.windows.net/data4fasttext/THUCNews.txt']
    data = Dataset.File.from_files(path=path)
    data.register(workspace=workspace, name=dataset_name, description='THUCNews dataset is generated by filtering and filtering historical data of Sina News RSS subscription channel from 2005 to 2011')
    print('Registerd')
data = workspace.datasets[dataset_name]

if char2index_name not in workspace.datasets:
    print('Registering a Char2Index_JSON for fasttext pipeline ...')
    path = ['https://datastore4fasttext.file.core.windows.net/data4fasttext/character2index.json']
    data = Dataset.File.from_files(path=path)
    data.register(workspace=workspace, name=char2index_name, description='The mapping relationship between character and index ')
    print('Registerd')
char2index = workspace.datasets[char2index_name]

print(data)
print(char2index)

In [ ]:
# load module
try:
    split_data_txt_module_func = Module.load(workspace=workspace, namespace=namespace, name='Split Data Txt')
    print('found split_data_txt_module')
except:
    print('not found split_data_txt_module, register it now...')
    yaml_file='split_data_txt/split_data_txt.spec.yaml'
    split_data_txt_module_func = Module.register(workspace=workspace, yaml_file=yaml_file)

try:
    split_data_txt_parallel_module_func = Module.load(workspace=workspace, namespace=namespace, name='Split Data Txt Parallel')
    print('found split_data_txt_parallel_module')
except:
    print('not found split_data_txt_parallel_module, register it now...')
    yaml_file='split_data_txt_parallel/split_data_txt_parallel.spec.yaml'
    split_data_txt_parallel_module_func = Module.register(workspace=workspace, yaml_file=yaml_file)

try:
    fasttext_train_module_func = Module.load(workspace=workspace, namespace=namespace, name='FastText Train')
    print('found fasttext_train_module')
except:
    print('not found fasttext_train_module, register it now...')
    yaml_file='fasttext_train/fasttext_train.spec.yaml'
    fasttext_train_module_func = Module.register(workspace=workspace, yaml_file=yaml_file)

try:
    fasttext_evaluation_module_func = Module.load(workspace=workspace, namespace=namespace, name='FastText Evaluation')
    print('found fasttext_evaluation_module')
except:
    print('not found fasttext_evaluation_module, register it now...')
    yaml_file='fasttext_evaluation/fasttext_evaluation.spec.yaml'
    fasttext_evaluation_module_func = Module.register(workspace=workspace, yaml_file=yaml_file)

try:
    fasttext_score_module_func = Module.load(workspace=workspace, namespace=namespace, name='FastText Score')
    print('found fasttext_score_module')
except:
    print('not found fasttext_score_module, register it now...')
    yaml_file='fasttext_score/fasttext_score.spec.yaml'
    fasttext_score_module_func = Module.register(workspace=workspace, yaml_file=yaml_file)

try:
    fasttext_score_parallel_module_func = Module.load(workspace=workspace, namespace=namespace, name='FastText Score Parallel')
    print('found fasttext_score_parallel_module')
except:
    print('not found fasttext_score_parallel_module, register it now...')
    yaml_file='fasttext_score_parallel/fasttext_score_parallel.spec.yaml'
    fasttext_score_parallel_module_func = Module.register(workspace=workspace, yaml_file=yaml_file)

try:
    compare_two_models_module_func = Module.load(workspace=workspace, namespace=namespace, name='Compare Two Models')
    print('found compare_two_models_module')
except:
    print('not found compare_two_models_module, register it now...')
    yaml_file='compare_two_models/compare_two_models.spec.yaml'
    compare_two_models_module_func = Module.register(workspace=workspace, yaml_file=yaml_file)



# inspect signature
# print(inspect.signature(split_data_txt_module_func))
# print(inspect.signature(split_data_txt_parallel_module_func))
# print(inspect.signature(fasttext_train_module_func))
# print(inspect.signature(fasttext_evaluation_module_func))
# print(inspect.signature(fasttext_score_module_func))
# print(inspect.signature(fasttext_score_parallel_module_func))
# print(inspect.signature(compare2model_module_func))

In [ ]:
# connect module
@dsl.pipeline(name='test deploy', description='Test parallel', default_compute_target=aml_compute_name)
def training_pipeline(epochs):
    split_data_txt_parallel = split_data_txt_parallel_module_func(
    input_dir = data,
    training_data_ratio = 0.7,
    validation_data_ratio = 0.1,
    random_split = True,
    seed = 9
    )

    fasttext_train = fasttext_train_module_func(
    training_data_dir = split_data_txt_parallel.outputs.training_data_output,
    validation_data_dir = split_data_txt_parallel.outputs.validation_data_output,
    char2index_dir = char2index,
    epochs = epochs,
    batch_size = 64,
    learning_rate = 0.0005,
    embedding_dim = 128
    )
    
    fasttext_score_parallel = fasttext_score_parallel_module_func(
    texts_to_score = split_data_txt_parallel.outputs.test_data_output,
    fasttext_model = fasttext_train.outputs.trained_model_dir,
    char2index_dir = char2index
    )
    fasttext_score_parallel.runsettings.configure(node_count=4, process_count_per_node=8, mini_batch_size=128)

    return {**fasttext_score_parallel.outputs, **fasttext_train.outputs}


In [ ]:
split_data_txt_parallel = split_data_txt_parallel_module_func(
    input_dir = data,
    training_data_ratio = 0.7,
    validation_data_ratio = 0.1,
    random_split = True,
    seed = 8
    )

fasttext_train = fasttext_train_module_func(
    training_data_dir = split_data_txt_parallel.outputs.training_data_output,
    validation_data_dir = split_data_txt_parallel.outputs.validation_data_output,
    char2index_dir = char2index,
    epochs = 1,
    batch_size = 64,
    learning_rate = 0.0005,
    embedding_dim = 128
    )
type(fasttext_train)

In [ ]:
# pipeline
pipeline = training_pipeline(epochs=1)

In [ ]:
# visualization
pipeline.validate()

In [ ]:
# pipeline_draft = pipeline.save(
#     experiment_name='my test',
# )
# pipeline_draft

In [ ]:
# run
run = pipeline.submit(experiment_name='parallel', pipeline_parameters={'epochs':1})
run.wait_for_completion()
run
#%


In [ ]:
type(run)

In [ ]:
# model = run.register_model(model_name='fasttext_model',
#                            model_path='Trained_model_dir') #Trained_model_dir/BestModel
# print(model.name, model.id, model.version, sep='\t')

pipeline.outputs.trained_model_dir.AVAILABLE_MODE



type(run)




In [ ]:
# model = run.register_model(model_name='fasttext_model',
#                            model_path='Trained_model_dir') #Trained_model_dir/BestModel

In [ ]:
# run.pipeline.outputs.trained_model_dir.configure()

In [ ]:
aaa=[]
for cr in run.get_children():
    aaa.append(cr)
    print(cr,'\n')

In [ ]:
# run.child_run(name='FastText Train').register_model(model_name='BestModel', model_path='Trained_model_dir') #Trained_model_dir/BestModel
run.get_file_names()

In [ ]:
aaa[-2].get_file_names()

In [ ]:
type(aaa[-2])

In [ ]:
# aaa[-2].register_model(model_name='BestModel', model_path='Trained_model_dir', tags={'my_tags': 'fasttext'})

In [ ]:
run.get_children()

In [ ]:
# register env for deployment to workspace
# from azureml.core.environment import Environment
# myenv = Environment.from_conda_specification(name = 'env_for_deployment',
#                                              file_path = 'deployment/env_for_deployment.yaml')
# myenv.register(workspace=workspace)

In [ ]:
# from azureml.core.model import Model
# model = Model.get_model_path(model_name='fasttext', version=2)
# Model.deploy(workspace=workspace, name='my_deployment', models=[model], inference_config=None, deployment_config=None, deployment_target=None, overwrite=False)

In [ ]:
# 3. Define inference configuration
# from azureml.core.environment import Environment
# from azureml.core.model import InferenceConfig


# myenv = Environment.get(workspace=workspace, name='env_for_deployment', version='2')
# inference_config = InferenceConfig(entry_script='deployment/scoring_for_deployment.py',
#                                    environment=myenv)
# inference_config                                   

In [ ]:
# 创建k8s资源
# from azureml.core.compute import AksCompute, ComputeTarget

# # Use the default configuration (you can also provide parameters to customize this).
# # For example, to create a dev/test cluster, use:
# # prov_config = AksCompute.provisioning_configuration(cluster_purpose = AksCompute.ClusterPurpose.DEV_TEST)
# prov_config = AksCompute.provisioning_configuration()
# aks_name = 'myaks'
# # Create the cluster
# aks_target = ComputeTarget.create(workspace = workspace,
#                                     name = aks_name,
#                                     provisioning_configuration = prov_config)

# # Wait for the create process to complete
# aks_target.wait_for_completion(show_output = True)


In [ ]:
# # Attach an existing AKS cluster
# from azureml.core.compute import AksCompute, ComputeTarget
# # Set the resource group that contains the AKS cluster and the cluster name
# resource_group = 'myresourcegroup'
# cluster_name = 'myexistingcluster'

# # Attach the cluster to your workgroup. If the cluster has less than 12 virtual CPUs, use the following instead:
# # attach_config = AksCompute.attach_configuration(resource_group = resource_group,
# #                                         cluster_name = cluster_name,
# #                                         cluster_purpose = AksCompute.ClusterPurpose.DEV_TEST)
# attach_config = AksCompute.attach_configuration(resource_group = resource_group,
#                                          cluster_name = cluster_name)
# aks_target = ComputeTarget.attach(ws, 'myaks', attach_config)

In [ ]:
#####################################################
# register env for deployment to workspace
from azureml.core.environment import Environment
myenv = Environment.from_conda_specification(name = 'env_for_deployment',
                                             file_path = 'deployment/env_for_deployment.yaml')
myenv.register(workspace=workspace)



In [ ]:

from azureml.core.model import Model
# model = Model.get_model_path(model_name='BestModel', version=2)
Model.list(workspace)
# Model.deploy(workspace=workspace, name='my_deployment', models=[model], inference_config=None, deployment_config=None, deployment_target=None, overwrite=False)
    

In [ ]:
# model = Model.get_model_path(model_name='BestModel', version=2, _workspace=workspace)  # 会把模型下载到本地的azureml-models目录下
# model

In [ ]:
# 3. Define inference configuration
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig


myenv = Environment.get(workspace=workspace, name='env_for_deployment', version='2')
inference_config = InferenceConfig(entry_script='deployment/scoring_for_deployment.py',
                                   environment=myenv)
inference_config      

In [ ]:
model = aaa[-2].register_model(model_name='BestModel', model_path='Trained_model_dir', tags={'my_tags': 'fasttext'})
model

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(workspace=workspace, name="my-deployment", models=[model], inference_config=inference_config, deployment_config=deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(workspace=workspace, name="my-deployment", models=[model], inference_config=inference_config, deployment_config=deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 2)
service = Model.deploy(workspace=workspace, name="my-deployment2", models=[model], inference_config=inference_config, deployment_config=deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 2)
service = Model.deploy(workspace=workspace, name="my-deployment3", models=[model], inference_config=inference_config, deployment_config=deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

In [ ]:
import requests
import json

In [ ]:
headers = {'Content-Type': 'application/json'}

if service.auth_enabled:
    headers['Authorization'] = 'Bearer '+service.get_keys()[0]
elif service.token_auth_enabled:
    headers['Authorization'] = 'Bearer '+service.get_token()[0]

print(headers)


In [ ]:
service.auth_enabled

In [ ]:

# test_sample = json.dumps({'data': [
#     [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
#     [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
# ]})

standard_sample_input = {'input sentence': 'i want to travel around the world'}

response = requests.post(
    service.scoring_uri, data=standard_sample_input, headers=headers)
print(response.status_code)
print(response.elapsed)
print(response.json())

In [ ]:
service

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies


environment = Environment('my-sklearn-environment')
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults',
    'inference-schema[numpy-support]',
    'joblib',
    'numpy',
    'scikit-learn'
])

In [ ]:
# register env for deployment to workspace
from azureml.core.environment import Environment
myenv = Environment.from_conda_specification(name = 'env_for_deployment',
                                             file_path = 'deployment/env_for_deployment.yaml')
myenv.register(workspace=workspace)

In [ ]:
from azureml.core import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.exceptions import WebserviceException


service_name = 'my-service-1'

# Remove any existing service under the same name.
try:
    Webservice(workspace, service_name).delete()
except WebserviceException:
    pass

inference_config = InferenceConfig(entry_script='deployment/scoring_for_deployment.py', environment=myenv)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=workspace,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config)
service.wait_for_deployment(show_output=True)